In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras import losses
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
from sklearn.metrics import log_loss
from glob import glob
import csv  

In [2]:
#Parsing all the t
path = '/kaggle/input/csc4851-homework4/birds_400/train/'
train_datagen = ImageDataGenerator(rescale=1. / 224)
train = train_datagen.flow_from_directory(path, target_size=(224,224), class_mode='categorical')

classes = [x[0].split('/')[-1] for x in os.walk(path)]
classes.pop(0)

Found 58388 images belonging to 400 classes.


''

In [3]:
# Using alexnet
def AlexNet(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = BatchNormalization(axis = 3 , name = "bn0")(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
    X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = BatchNormalization(axis = 3 ,name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
    X = Flatten()(X)
    
    X = Dense(4096, activation = 'relu', name = "fc0")(X)
    
    X = Dense(4096, activation = 'relu', name = 'fc1')(X) 
    
    X = Dense(400,activation='softmax',name = 'fc2')(X)
    model = Model(inputs = X_input, outputs = X, name='AlexNet')
    
    return model

In [4]:
alex = AlexNet(train[0][0].shape[1:])

2022-03-28 15:44:24.866547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-28 15:44:24.960496: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-28 15:44:24.961240: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-28 15:44:24.962361: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
alex.summary()

Model: "AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv0 (Conv2D)               (None, 54, 54, 96)        34944     
_________________________________________________________________
bn0 (BatchNormalization)     (None, 54, 54, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 54, 54, 96)        0         
_________________________________________________________________
max0 (MaxPooling2D)          (None, 26, 26, 96)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 26, 26, 256)       614656    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 26, 26, 256)       1024

In [6]:
alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [7]:
alex.fit_generator(train,epochs=7)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-03-28 15:44:27.339787: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/7


2022-03-28 15:44:29.190024: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1825/1825 [==============================] - 364s 196ms/step - loss: 5.4494 - accuracy: 0.0226
Epoch 2/7
1825/1825 [==============================] - 136s 74ms/step - loss: 4.1734 - accuracy: 0.1150
Epoch 3/7
1825/1825 [==============================] - 142s 78ms/step - loss: 3.3625 - accuracy: 0.2331
Epoch 4/7
1825/1825 [==============================] - 145s 79ms/step - loss: 2.7851 - accuracy: 0.3359
Epoch 5/7
1825/1825 [==============================] - 138s 75ms/step - loss: 2.3156 - accuracy: 0.4313
Epoch 6/7
1825/1825 [==============================] - 137s 75ms/step - loss: 1.9578 - accuracy: 0.5054
Epoch 7/7
1825/1825 [==============================] - 139s 76ms/step - loss: 1.6651 - accuracy: 0.5713


In [8]:
path_test = '/kaggle/input/csc4851-homework4/birds_400/test'
test_datagen = ImageDataGenerator(rescale=1. / 224)
test = test_datagen.flow_from_directory(path_test, target_size=(224,224), class_mode='categorical')

Found 2000 images belonging to 400 classes.


In [9]:
preds = alex.evaluate_generator(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100))

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Loss = 1.566264271736145
Test Accuracy = 59.20000076293945


In [10]:
#validation

path_test = '/kaggle/input/csc4851-homework4/birds_400/valid'
validation_datagen = ImageDataGenerator(rescale=1. / 224)
validation = validation_datagen.flow_from_directory(path_test, target_size=(224,224), batch_size = 1,class_mode='categorical')

Found 2000 images belonging to 400 classes.


In [11]:
preds = alex.evaluate_generator(validation)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100))

Loss = 1.7701237201690674
Test Accuracy = 57.59999752044678


In [12]:
def prepare(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = x/224
    return np.expand_dims(x, axis=0)

In [13]:
cce = losses.CategoricalCrossentropy()

In [16]:
# result = alex.predict([prepare("/kaggle/input/csc4851-homework4/birds_400/test/ABBOTTS BABBLER/1.jpg")])
# print(len(result[0]))
template = [0]*400
log_loss = {}
test_images = glob("/kaggle/input/csc4851-homework4/birds_400/test/" + "*/*.jpg")

for path in test_images:
    result = alex.predict([prepare(path)])
    actual_class = path.split('/')[-2]
    actual_class = "BLACK & YELLOW  BROADBILL" if actual_class == "BLACK & YELLOW BROADBILL" else actual_class
    actual_index = classes.index(actual_class)
    template[actual_index] = 1
    log_loss_current = cce(template, result[0]).numpy()
#     print(log_loss_current)
    if actual_index in log_loss:
        log_loss[actual_index] += (log_loss_current)/100
    else:
        log_loss[actual_index] = (log_loss_current)/100
    template[actual_index] = 0

# print(template)
# print(classes[int(x)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
# print(log_loss)

{0: 0.6994052219390869, 1: 0.7882149887084962, 2: 0.6326978778839112, 3: 0.7842756843566894, 4: 0.7625520992279053, 5: 0.805904769897461, 6: 0.7118411445617676, 7: 0.805904769897461, 8: 0.805904769897461, 9: 0.7412753009796142, 10: 0.805904769897461, 11: 0.31053852558135986, 12: 0.6157006788253784, 13: 0.7893287658691406, 14: 0.805904769897461, 15: 0.805904769897461, 16: 0.805904769897461, 17: 0.7632537460327149, 18: 0.805904769897461, 19: 0.7039810371398926, 20: 0.805904769897461, 21: 0.7802321147918702, 22: 0.805904769897461, 23: 0.7207791423797607, 24: 0.805904769897461, 25: 0.35284067630767824, 26: 0.728964786529541, 27: 0.805904769897461, 28: 0.805904769897461, 29: 0.7440276527404784, 30: 0.805904769897461, 31: 0.7580771636962891, 32: 0.4206940484046936, 33: 0.7784725761413573, 34: 0.32776348114013676, 35: 0.7841670131683349, 36: 0.5720710420608521, 37: 0.8011806392669678, 38: 0.6084196853637696, 39: 0.805904769897461, 40: 0.7972394561767578, 41: 0.805904769897461, 42: 0.797519350

In [29]:
print('Generating submission.csv file...')
ids = list(log_loss.keys())
values = list(log_loss.values())

f = open('submission.csv', 'w')
writer = csv.writer(f)
writer.writerow(['id','birds'])
for index in range(len(ids)):
    writer.writerow([ids[index],values[index]])

!head submission.csv


Generating submission.csv file...
id,birds
0,0.6994052219390869
1,0.7882149887084962
2,0.6326978778839112
3,0.7842756843566894
4,0.7625520992279053
5,0.805904769897461
6,0.7118411445617676
7,0.805904769897461
8,0.805904769897461
